### 주차별 프로젝트(1주차 : 종이 헬리콥터 회귀분석)

구성 변수 : 날개 길이, 날개 폭, 다리 길이, 몸통 길이 + ɑ

모델 구성 : scikit-learn, NN

### 데이터 구성
실제로 만들어서 시간을 관측하고, 시간을 측정하는 기준은 아래와 같다.

- 두 날개의 가장 안쪽을 두 손가락을 이용해 천장에 붙인 뒤 낙하

In [30]:
import pandas as pd

dataset = {
    "wing_length" : [11.0, 9.5, 10.2, 16.0, 9.5, 13.25, 13.25, 11.8, 8.5, 8.0, 9.7, 9.0, 9.0, 9.0, 7.5],
    "wing_width" : [6.25, 7.4, 5.3, 7.4, 7.4, 7.45, 7.45, 6.95, 2.7, 2.7, 3.0, 3.0, 3.0, 3.0, 2.7],
    "body_length" : [3.0, 3.0, 1.8, 1.0, 3.0, 2.0, 2.0, 2.25, 2.0, 2.0, 3.0, 3.0, 2.5, 2.15, 1.65],
    "leg_length" : [7.05, 8.5, 2.5, 4.0, 3.6, 4.65, 3.8, 4.8, 2.5, 2.5, 3.5, 3.5, 4.0, 4.3, 2.8],
    "target" : [2.31, 1.77, 2.73, 2.43, 2.27, 2.5, 2.28, 2.41, 2.41, 2.76, 2.98, 2.69, 2.73, 2.8, 2.83]
}

len(dataset['wing_length'])
df = pd.DataFrame(dataset, columns = dataset.keys(), index = list(range(len(dataset['wing_length']))))
df.shape

(15, 5)

### 학습 데이터 구성

In [31]:
X = df.drop(labels = 'target', axis = 1)
y = df[['target']]

In [32]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0, 1))
X = scaler.fit_transform(X)
X

array([[0.41176471, 0.74736842, 1.        , 0.75833333],
       [0.23529412, 0.98947368, 1.        , 1.        ],
       [0.31764706, 0.54736842, 0.4       , 0.        ],
       [1.        , 0.98947368, 0.        , 0.25      ],
       [0.23529412, 0.98947368, 1.        , 0.18333333],
       [0.67647059, 1.        , 0.5       , 0.35833333],
       [0.67647059, 1.        , 0.5       , 0.21666667],
       [0.50588235, 0.89473684, 0.625     , 0.38333333],
       [0.11764706, 0.        , 0.5       , 0.        ],
       [0.05882353, 0.        , 0.5       , 0.        ],
       [0.25882353, 0.06315789, 1.        , 0.16666667],
       [0.17647059, 0.06315789, 1.        , 0.16666667],
       [0.17647059, 0.06315789, 0.75      , 0.25      ],
       [0.17647059, 0.06315789, 0.575     , 0.3       ],
       [0.        , 0.        , 0.325     , 0.05      ]])

In [33]:
y = scaler.fit_transform(y)
y

array([[0.44628099],
       [0.        ],
       [0.79338843],
       [0.54545455],
       [0.41322314],
       [0.60330579],
       [0.4214876 ],
       [0.52892562],
       [0.52892562],
       [0.81818182],
       [1.        ],
       [0.76033058],
       [0.79338843],
       [0.85123967],
       [0.87603306]])

### 모델 빌드
- input layer's shape = (4, ) ➡️ 4개의 독립변수
- 64개의 노드를 가지는 hidden layer 4개층 구성, 활성화 함수는 relu 사용
- output layer's shape = (1, ) ➡️ 비행시간에 해당하는 1개의 출력 변수, 항등함수로 활성화

### scikit learn
다중 선형회귀 분석 모델들로 시도

In [34]:
from keras.models import Model
from keras.layers import Dense, Input
from keras.optimizers import Adam

input_layer = Input(shape = (4, ))
x = Dense(64, activation = "relu")(input_layer)
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
output_layer = Dense(1, activation = "sigmoid")(x)

optim = Adam(learning_rate = 0.001)

model = Model(input_layer, output_layer)
model.compile(optimizer = optim, loss = "mean_squared_error")

In [35]:
model.fit(X, y, epochs = 500)

Epoch 1/500
1/1 [==============================] - 0s 378ms/step - loss: 0.0760
Epoch 2/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0745
Epoch 3/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0732
Epoch 4/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0720
Epoch 5/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0707
Epoch 6/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0696
Epoch 7/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0684
Epoch 8/500
1/1 [==============================] - 0s 9ms/step - loss: 0.0672
Epoch 9/500
1/1 [==============================] - 0s 7ms/step - loss: 0.0659
Epoch 10/500
1/1 [==============================] - 0s 15ms/step - loss: 0.0644
Epoch 11/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0630
Epoch 12/500
1/1 [==============================] - 0s 8ms/step - loss: 0.0614
Epoch 13/500
1/1 [==============================] - 0s 7m

In [36]:
predicted_list = []
for i in range(X.shape[0]):
    predicted_list.append(model.predict(X)[i][0])

1/1 [==============================] - 0s 32ms/step


In [37]:
import numpy as np

predicted_list = np.array(predicted_list).reshape(-1, 1)
predicted_list = scaler.inverse_transform(predicted_list)

df['predicted_target'] = predicted_list

In [38]:
df

,wing_length,wing_width,body_length,leg_length,target,predicted_target
0,11.00,6.25,3.00,7.05,2.31,2.347244
1,9.50,7.40,3.00,8.50,1.77,1.780017
2,10.20,5.30,1.80,2.50,2.73,2.741468
3,16.00,7.40,1.00,4.00,2.43,2.459759
4,9.50,7.40,3.00,3.60,2.27,2.290850
5,13.25,7.45,2.00,4.65,2.50,2.526776
6,13.25,7.45,2.00,3.80,2.28,2.312957
7,11.80,6.95,2.25,4.80,2.41,2.449284
8,8.50,2.70,2.00,2.50,2.41,2.432340
9,8.00,2.70,2.00,2.50,2.76,2.759758


In [44]:
import numpy as np

random_data = np.random.random(15).reshape(-1, 1)
scaler.inverse_transform(random_data)
# model.predict()

array([[1.99249376],
       [1.79178192],
       [2.81757337],
       [2.31025661],
       [1.80440544],
       [2.8330092 ],
       [1.93009213],
       [2.31223886],
       [2.24972639],
       [2.43402756],
       [2.72656053],
       [1.86338019],
       [2.72380829],
       [2.9166499 ],
       [2.4604187 ]])